In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [4]:
from ai.datamodules.text import TextDataModule
from ai.constants import REPO_DIR
from ai.misc_utils import get_vocab_size
import matplotlib.pyplot as plt
import numpy as np
import torch
import math
from torch import nn
import torch.nn.functional as F
from dataclasses import dataclass
from ai.models.nlp.transformer import Transformer as Transformer2
fname = f'{REPO_DIR}/data/shakespeare.txt'
# fname = f'{REPO_DIR}/data/names.txt'

In [5]:
config = Transformer2.Config(
    vocab_size = get_vocab_size(fname),
    block_size = 32,
    # mlp_size:int = 64
    n_embd = 128,
    n_heads = 16,
    n_blocks = 4,
    dropout=.2
)
batch_size = 4096

In [6]:
dm = TextDataModule.Config(fname=fname, block_size=config.block_size, batch_size=batch_size, 
                           num_workers=0).i()
dm.prepare_data()

2024-02-09 09:37:12.951 | INFO     | ai.datamodules.text:prepare_data:76 - Creating datasets...


In [25]:
class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed = nn.Embedding(config.vocab_size, config.n_embd)
        self.transformer = nn.Transformer(d_model=config.n_embd, nhead=config.n_heads, num_encoder_layers=0, num_decoder_layers=config.n_blocks, 
                           dropout=config.dropout,
                            batch_first=True, dim_feedforward=config.n_embd*4)      
        self.proj = nn.Linear(config.n_embd, config.vocab_size)

    def forward(self,x,y):
        o = self.embed(x)
        o = self.transformer(o,y)
        return self.proj(o)
        

In [27]:
# B = batch_size
T = config.block_size
# model = Transformer2(config)
model = Transformer(config)
model = model.cuda()
dl = dm.train_dataloader()

In [10]:
lr=5e-3
step=0

for epoch in range(40):
    for X,y in iter(dl):
        X = X.cuda()
        y = y.cuda()
        step+=1
        B = X.shape[0]

        # forward
        logits = model(X)
        loss = F.cross_entropy(logits.view(B*T,config.vocab_size), y.view(B*T))
        # logits = model(X)
        # loss = F.cross_entropy(logits.view(B*T,config.vocab_size), y.view(B*T))
        if step % 100 == 0:
            print(f'e{epoch}|s{step}: loss={loss:.2f}')

        for param in model.parameters():
            param.grad = None
        loss.backward()
        for param in model.parameters():
            if param.grad is not None:
                param.data -= param.grad * lr


TypeError: Transformer.forward() missing 1 required positional argument: 'tgt'

In [ ]:
x = X.clone()
result = []
for i in range(30):
    with torch.inference_mode():
        logits = model(x)
    probas = torch.softmax(logits, -1)
    # greedy
    # token_ids = probas[:,-1].argmax(-1)
    token_ids = torch.multinomial(probas[:,-1], num_samples=1)
    x = x.roll(-1)
    x[:,-1] = token_ids.flatten()
    result.append(token_ids.flatten())


prompt = X[0]
answer = result[0]
print(''.join(dm.ds_train.decode(prompt)))
print('*'*72)
print(''.join(dm.ds_train.decode(answer)))

In [20]:
probas.shape

torch.Size([256, 32, 65])

In [22]:
# dm.ds_train.decode(answer)